In [1]:
import sys
sys.path.append("./work")

In [2]:
raw_data_path = "./work/data/knmi.parquet"
output_directory_path = "./work/output/"
considered_cities = ["De Bilt"]
start_date = "2003-01-01"

In [3]:
from pyspark.sql import SparkSession

In [4]:
spark = SparkSession.builder.appName("HeatwaveAnalysis").getOrCreate()

In [5]:
df = spark.read.parquet(raw_data_path, header=True, inferSchema=True)

In [6]:
import src.common as c
import src.heatwaves as heat


df = c.filter_needed_data(df, considered_cities, start_date)
df = c.clean_data(df)
df = c.with_date(df)
df = c.aggregate_daily(df)
df = heat.with_maximum_flags(df)
df = heat.filter_wave_candidates(df)
df = c.with_prev_date(df)
df = c.with_date_diff(df)
df = c.with_group(df)
df = heat.group_metrics(df)
df = heat.filter_heatwaves(df)

In [7]:
from pyspark.sql import functions as F


(df
 .select(
    F.col("FROM_DATE"),
    F.col("TO_DATE"),
    F.col("DURATION"),
    F.col("TROPICAL_DAYS"),
    F.col("MAX_TEMPERATURE"),
)
    .orderBy(F.col("FROM_DATE"))
    .coalesce(1)
    .write
    .option("header", "true")
    .csv(output_directory_path + "heatwaves.csv")
)